In [1]:
import pandas as pd
import numpy as np



In [42]:
data = pd.read_csv('data_with_betting_odds_v1.csv', delimiter=';')
data['Betting Predicition'] = ''
data.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,B365H,B365D,B365A,BWH,...,WHH,WHD,WHA,SJH,SJD,SJA,VCH,VCD,VCA,Betting Predicition
0,11.08.2006,Bayern Munich,Dortmund,2,0,H,157,36,60,155,...,153.0,35.0,55.0,144.0,375.0,70.0,145.0,375.0,60.0,
1,12.08.2006,Hamburg,Bielefeld,1,1,D,144,40,75,135,...,14.0,375.0,70.0,144.0,36.0,75.0,14.0,375.0,70.0,
2,12.08.2006,Leverkusen,Aachen,3,0,H,144,40,75,15,...,144.0,36.0,65.0,14.0,41.0,70.0,145.0,36.0,65.0,
3,12.08.2006,Mainz,Bochum,2,1,H,20,32,375,185,...,195.0,31.0,35.0,183.0,34.0,38.0,19.0,32.0,35.0,
4,12.08.2006,M'gladbach,Cottbus,2,0,H,161,36,55,16,...,16.0,34.0,48.0,157.0,36.0,55.0,153.0,35.0,55.0,


In [39]:
betting_providers = ['B365H','B365D','B365A','BWH','BWD','BWA','IWH','IWD','IWA','VCH','VCD','VCA']
# 'GBH','GBD','GBA', 'WHH','WHD','WHA', 'SBH','SBD', 'SBA', 'SJH','SJD','SJA', 'LBH','LBD','LBA',


In [33]:
def odds_to_probaility(odds):
    if odds >= 0:
        probability = odds/(odds+100)
    else:
        probability = odds/(odds-100)
    return probability
print(odds_to_probaility(data[betting_providers[1]][0]))

0.2647058823529412


In [45]:
x,z = 0,0
prob_spieltag, prob_ges = [], []
while x < len(data):
    while z < len(betting_providers):
        prob = odds_to_probaility(data[betting_providers[z]][x])
        prob_spieltag.append(prob)
        print(data[betting_providers[z]][x])
        print((prob))
        z +=1
    prob_ges.append(prob_spieltag)
    prob_spieltag = []
    z=0
    x+=1


157
0.6108949416342413
36
0.2647058823529412
60
0.375
155
0.6078431372549019
405
0.801980198019802
475
0.8260869565217391
155.0
0.6078431372549019
37.0
0.27007299270072993
50.0
0.3333333333333333
145.0
0.5918367346938775
375.0
0.7894736842105263
60.0
0.375
144
0.5901639344262295
40
0.2857142857142857
75
0.42857142857142855
135
0.574468085106383
44
0.3055555555555556
75
0.42857142857142855
14.0
0.12280701754385964
40.0
0.2857142857142857
65.0
0.3939393939393939
14.0
0.12280701754385964
375.0
0.7894736842105263
70.0
0.4117647058823529
144
0.5901639344262295
40
0.2857142857142857
75
0.42857142857142855
15
0.13043478260869565
38
0.2753623188405797
58
0.3670886075949367
145.0
0.5918367346938775
39.0
0.2805755395683453
60.0
0.375
145.0
0.5918367346938775
36.0
0.2647058823529412
65.0
0.3939393939393939
20
0.16666666666666666
32
0.24242424242424243
375
0.7894736842105263
185
0.6491228070175439
33
0.24812030075187969
39
0.2805755395683453
20.0
0.16666666666666666
32.0
0.24242424242424243
33.0
0

In [47]:
prob_ges

[[0.6108949416342413,
  0.2647058823529412,
  0.375,
  0.6078431372549019,
  0.801980198019802,
  0.8260869565217391,
  0.6078431372549019,
  0.27007299270072993,
  0.3333333333333333,
  0.5918367346938775,
  0.7894736842105263,
  0.375],
 [0.5901639344262295,
  0.2857142857142857,
  0.42857142857142855,
  0.574468085106383,
  0.3055555555555556,
  0.42857142857142855,
  0.12280701754385964,
  0.2857142857142857,
  0.3939393939393939,
  0.12280701754385964,
  0.7894736842105263,
  0.4117647058823529],
 [0.5901639344262295,
  0.2857142857142857,
  0.42857142857142855,
  0.13043478260869565,
  0.2753623188405797,
  0.3670886075949367,
  0.5918367346938775,
  0.2805755395683453,
  0.375,
  0.5918367346938775,
  0.2647058823529412,
  0.3939393939393939],
 [0.16666666666666666,
  0.24242424242424243,
  0.7894736842105263,
  0.6491228070175439,
  0.24812030075187969,
  0.2805755395683453,
  0.16666666666666666,
  0.24242424242424243,
  0.24812030075187969,
  0.15966386554621848,
  0.24242424

In [81]:
len(data)
len(prob_ges)
len(prob_ges_schnitt)

4896

In [80]:
z, home, draw, away = 0, 0, 0, 0
prob_spieltag_schnitt, prob_ges_schnitt = [], []
for x in prob_ges:
    while z < len(x):
        if z % 3 == 0:
            home = home + x[z]
        if z % 3 == 1:
            draw = draw + x[z]
        if z % 3 == 2:
            away = away + x[z]
        z += 1
    z = 0
    prob_spieltag_schnitt.append(home/4)
    prob_spieltag_schnitt.append(draw/4)
    prob_spieltag_schnitt.append(away/4)
    print(len(prob_spieltag_schnitt))
    prob_ges_schnitt.append(prob_spieltag_schnitt)
    prob_spieltag_schnitt = []
    print(len(prob_ges_schnitt))
    print(home/4)
    print('')
    home, draw, away = 0, 0, 0


3
1
0.6046044877094807

3
2
0.3525615136550829

3
3
0.47606804660567004

3
4
0.2855300014742739

3
5
0.37436584737934064

3
6
0.47606804660567004

3
7
0.1596341926601149

3
8
0.4011499336576736

3
9
0.32143362555023425

3
10
0.40120354361840427

3
11
0.5466358639791014

3
12
0.3997857214757584

3
13
0.37326679797356227

3
14
0.16316526610644255

3
15
0.27425675015062334

3
16
0.25946331345605395

3
17
0.4537805853017983

3
18
0.46312409183376924

3
19
0.45129646298660975

3
20
0.3486375513251213

3
21
0.5243577741142699

3
22
0.38639818530218084

3
23
0.3651373592915427

3
24
0.5070911103681709

3
25
0.1836598693855791

3
26
0.2716564481270364

3
27
0.17352549338391365

3
28
0.4043933362818812

3
29
0.32473875511131306

3
30
0.3407864294961069

3
31
0.43964978111319575

3
32
0.1836598693855791

3
33
0.4944444167113632

3
34
0.4852523990479291

3
35
0.3448249246823798

3
36
0.45592912312585193

3
37
0.44796335161561773

3
38
0.4389478764478765

3
39
0.33275206796333556

3
40
0.327964561

In [67]:
prob_ges_schnitt

[[0.6046044877094807, 0.5315581893209999, 0.4773550724637681],
 [0.3525615136550829, 0.4166144527986633, 0.415711739241151],
 [0.47606804660567004, 0.276589506619038, 0.3911498575264398],
 [0.2855300014742739, 0.24384825700615176, 0.3943571959475027],
 [0.37436584737934064, 0.2619825708061002, 0.3451017431093333],
 [0.47606804660567004, 0.40906614361991933, 0.38528154690789096],
 [0.1596341926601149, 0.24384825700615176, 0.53234188412207],
 [0.4011499336576736, 0.37724544220316136, 0.4043933362818812],
 [0.32143362555023425, 0.3686857074376618, 0.4676477060085799],
 [0.40120354361840427, 0.37582142762125204, 0.41014828824426347],
 [0.5466358639791014, 0.24667503222098666, 0.521167981450793],
 [0.3997857214757584, 0.37441866698832643, 0.40614403656199316],
 [0.37326679797356227, 0.3950150742437394, 0.3451199338296112],
 [0.16316526610644255, 0.24242424242424243, 0.522666919302839],
 [0.27425675015062334, 0.37441866698832643, 0.5529701094437817],
 [0.25946331345605395, 0.2605594869526758

In [82]:
erg_odds = []
for x in prob_ges_schnitt:
    max_value = max(x)
    if x.index(max_value) == 0:
        erg_odds.append('H')
    if x.index(max_value) == 1:
        erg_odds.append('D')
    if x.index(max_value) == 2:
        erg_odds.append('A')
erg_odds

['H',
 'D',
 'H',
 'A',
 'H',
 'H',
 'A',
 'A',
 'A',
 'A',
 'H',
 'A',
 'D',
 'A',
 'A',
 'A',
 'H',
 'H',
 'H',
 'A',
 'H',
 'A',
 'D',
 'H',
 'A',
 'A',
 'D',
 'D',
 'D',
 'A',
 'H',
 'D',
 'H',
 'A',
 'D',
 'D',
 'A',
 'A',
 'D',
 'D',
 'H',
 'D',
 'A',
 'D',
 'D',
 'H',
 'A',
 'A',
 'A',
 'D',
 'D',
 'A',
 'D',
 'A',
 'D',
 'H',
 'A',
 'D',
 'A',
 'A',
 'H',
 'A',
 'H',
 'H',
 'A',
 'H',
 'A',
 'D',
 'A',
 'A',
 'D',
 'A',
 'A',
 'A',
 'H',
 'A',
 'H',
 'H',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'D',
 'A',
 'D',
 'A',
 'A',
 'H',
 'D',
 'H',
 'A',
 'A',
 'D',
 'D',
 'A',
 'D',
 'D',
 'A',
 'A',
 'D',
 'D',
 'A',
 'A',
 'A',
 'A',
 'D',
 'D',
 'A',
 'H',
 'A',
 'H',
 'A',
 'D',
 'A',
 'D',
 'A',
 'H',
 'A',
 'D',
 'D',
 'D',
 'H',
 'H',
 'D',
 'A',
 'H',
 'H',
 'H',
 'A',
 'H',
 'D',
 'A',
 'D',
 'A',
 'A',
 'A',
 'A',
 'H',
 'A',
 'A',
 'A',
 'A',
 'D',
 'H',
 'D',
 'D',
 'H',
 'H',
 'D',
 'D',
 'H',
 'A',
 'A',
 'A',
 'D',
 'D',
 'H',
 'D',
 'D',
 'A',
 'A',
 'H',
 'A',
 'D'

In [83]:
data['Betting Predicition'] = erg_odds

In [84]:
data.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,B365H,B365D,B365A,BWH,...,WHH,WHD,WHA,SJH,SJD,SJA,VCH,VCD,VCA,Betting Predicition
0,11.08.2006,Bayern Munich,Dortmund,2,0,H,157,36,60,155,...,153.0,35.0,55.0,144.0,375.0,70.0,145.0,375.0,60.0,H
1,12.08.2006,Hamburg,Bielefeld,1,1,D,144,40,75,135,...,14.0,375.0,70.0,144.0,36.0,75.0,14.0,375.0,70.0,D
2,12.08.2006,Leverkusen,Aachen,3,0,H,144,40,75,15,...,144.0,36.0,65.0,14.0,41.0,70.0,145.0,36.0,65.0,H
3,12.08.2006,Mainz,Bochum,2,1,H,20,32,375,185,...,195.0,31.0,35.0,183.0,34.0,38.0,19.0,32.0,35.0,A
4,12.08.2006,M'gladbach,Cottbus,2,0,H,161,36,55,16,...,16.0,34.0,48.0,157.0,36.0,55.0,153.0,35.0,55.0,H


In [85]:
data.to_csv("../dataframe_with_odd_predictions.csv")

In [10]:
# drop_test = [
#     'MaxCH', 'MaxCD', 'MaxCA', 'AvgCH', 'AvgCD', 'AvgCA', 'B365C>2.5', 'B365C<2.5',
#     'PC>2.5', 'PC<2.5', 'MaxC>2.5', 'MaxC<2.5', 'AvgC>2.5', 'AvgC<2.5', 'AHCh',
#     'B365CAHH', 'B365CAHA', 'PCAHH', 'PCAHA', 'MaxCAHH', 'MaxCAHA', 'AvgCAHH', 'AvgCAHA'
# ]
# data.drop(columns=drop_test)

,Unnamed: 0,Unnamed: 0.1,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,...,BWCA,IWCH,IWCD,IWCA,WHCH,WHCD,WHCA,VCCH,VCCD,VCCA
0,0,0,D1,11/08/06,Bayern Munich,Dortmund,2,0,H,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,D1,12/08/06,Hamburg,Bielefeld,1,1,D,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2,D1,12/08/06,Leverkusen,Aachen,3,0,H,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3,D1,12/08/06,Mainz,Bochum,2,1,H,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,4,D1,12/08/06,M'gladbach,Cottbus,2,0,H,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4891,4891,301,D1,14/05/2022,Mainz,Ein Frankfurt,2,2,D,1,...,3.80,1.85,4.0,3.85,1.83,4.2,3.60,1.87,4.00,3.75
4892,4892,302,D1,14/05/2022,M'gladbach,Hoffenheim,5,1,H,3,...,3.00,2.05,4.3,3.05,2.05,4.2,3.00,2.10,4.20,3.00
4893,4893,303,D1,14/05/2022,Stuttgart,FC Koln,2,1,H,1,...,2.90,2.20,3.8,3.05,2.15,4.0,2.90,2.20,4.00,2.90
4894,4894,304,D1,14/05/2022,Union Berlin,Bochum,3,2,H,2,...,10.00,1.30,6.0,10.00,1.29,5.5,10.00,1.29,5.75,10.00


In [18]:
print(data.B365H[0])

157
